# Pull CV entries files; strip out extraneous entries
<div style="text-align: right">
<p>
    <img src="https://pcmdi.github.io/assets/PCMDI/199x65px-PCMDI-Logo-Text-rectangle.png"
         width="91"
         height="30"
         class="fixed-height-image"
         style="margin-right: 20px"
         title="Program for Climate Model Diagnosis and Intercomparison"
         alt="Program for Climate Model Diagnosis and Intercomparison"
    >&nbsp;
    <img src="https://pcmdi.github.io/assets/LLNL/212px-LLNLiconPMS286-WHITEBACKGROUND.png"
         width="30"
         height="30"
         class="fixed-height-image"
         style="margin-right: 20px"
         title="Lawrence Livermore National Laboratory"
         alt="Lawrence Livermore National Laboratory"
    >&nbsp;
    <img src="https://pcmdi.github.io/assets/DOE/459x127px-DOE-Logo_Color_TextOnly.png"
         width="108"
         height="30"
         class="fixed-height-image"
         style="margin-right: 20px"
         title="United States Department of Energy"
         alt="United States Department of Energy"
    >
</p>
</div>

**Summary**

This file pulls a CMIP6Plus_CVs/CMOR3.9.0-era CV files, strips out
extraneous entries and saves the files for local use

**Authors**

Paul J. Durack ([durack1](https://github.com/durack1); [PCMDI](https://pcmdi.llnl.gov/), [Lawrence Livermore National Laboratory](https://www.llnl.gov/))

**Notes**

PJD 15 Feb 2025 - initiated<br>
PJD 15 Feb 2025 - first prototype functioning<br>
PJD 16 Feb 2025 - add source_id contact and other info; Add DRS<br>
PJD 16 Feb 2025 - reconstructed DRS and filename templates to order relevant information<br>
PJD 18 Feb 2025 - updated to remove variant_info and variant_label from required_global_attributes<br>
PJD 18 Feb 2025 - updated to remove variant_label from DRS<br>
PJD 20 Feb 2025 - updated tracking_id -> tracking_id_prefix<br>
PJD 20 Feb 2025 - update MACA3-0 registration, augment with available variables<br>
PJD 24 Feb 2025 - updated source_version_number -> source_version<br>
PJD 27 Feb 2025 - update license info<br>
PJD  4 Mar 2025 - adding `gr` and `gr1` grid_labels ([#40](https://github.com/PCMDI/DRCDP/issues/40))<br>
PJD 28 May 2025 - updating `license_id` and `region` CVs to match CMOR 3.10+ changes ([#40](https://github.com/PCMDI/DRCDP/issues/43))<br>
PJD 28 May 2025 - updating `license["license_template"]` to match CMOR 3.10+ changes ([#40](https://github.com/PCMDI/DRCDP/issues/43))<br>
TODO:

**Links**

### imports

In [1]:
%%time
import datetime
import json
import os
import requests

CPU times: user 32.9 ms, sys: 12.8 ms, total: 45.7 ms
Wall time: 56.3 ms


### set table, coordinate, formula_terms and grids URLs and pull files

In [2]:
%%time
urls = {
    "frequency": "https://raw.githubusercontent.com/PCMDI/mip-cmor-tables/refs/heads/main/MIP_frequency.json",
    "grid_label": "https://raw.githubusercontent.com/PCMDI/mip-cmor-tables/refs/heads/main/MIP_grid_label.json",
    "license": "https://raw.githubusercontent.com/PCMDI/mip-cmor-tables/refs/heads/main/MIP_license.json",
    "nominal_resolution": "https://raw.githubusercontent.com/PCMDI/mip-cmor-tables/refs/heads/main/MIP_nominal_resolution.json",
    "realm": "https://raw.githubusercontent.com/PCMDI/mip-cmor-tables/refs/heads/main/MIP_realm.json",
    "source_type": "https://raw.githubusercontent.com/PCMDI/obs4MIPs-cmor-tables/refs/heads/master/obs4MIPs_source_type.json",
}
# create: institution_id, product, required_global_attributes, source_id, table_id, tracking_id

# loop through urls
keys = urls.keys()
for count, key in enumerate(keys):
    print(count, key)
    url = urls[key]
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4/5xx)
        vars()[key] = json.loads(response.text)
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON decode failed: {e}")
    except Exception as e:
        print(f"Unexpected error occurred: {e}")

0 frequency
1 grid_label
2 license
3 nominal_resolution
4 realm
5 source_type
CPU times: user 59.5 ms, sys: 21.4 ms, total: 80.8 ms
Wall time: 1.68 s


### create DRS

In [3]:
DRS = {}
DRS["DRS"] = {}
DRS["DRS"]["directory_path_template"] = "".join(
    [
        "<activity_id><region_id><institution_id><source_id><driving_mip_era>",
        "<driving_activity_id><driving_experiment_id><driving_source_id>",
        "<driving_variant_label><frequency><variable_id><version>",
    ]
)
DRS["DRS"][
    "directory_path_example"
] = "DRCDP/NAM/UCSD-SIO/LOCA2-1/CMIP6/CMIP/historical/ACCESS-CM2/r1i1p1f1/day/tasmax/v20250216"
DRS["DRS"]["filename_template"] = "".join(
    [
        "<variable_id><region_id><institution_id><source_id>",
        "<driving_mip_era><driving_experiment_id><driving_source_id>",
        "<driving_variant_label><frequency>",
    ]
)
DRS["DRS"][
    "filename_example"
] = "tasmax_NAM_UCSD-SIO_LOCA2-1_CMIP6_historical_ACCESS-CM2_r1i1p1f1_day_19500102-19500131.nc"
DRS

{'DRS': {'directory_path_template': '<activity_id><region_id><institution_id><source_id><driving_mip_era><driving_activity_id><driving_experiment_id><driving_source_id><driving_variant_label><frequency><variable_id><version>',
  'directory_path_example': 'DRCDP/NAM/UCSD-SIO/LOCA2-1/CMIP6/CMIP/historical/ACCESS-CM2/r1i1p1f1/day/tasmax/v20250216',
  'filename_template': '<variable_id><region_id><institution_id><source_id><driving_mip_era><driving_experiment_id><driving_source_id><driving_variant_label><frequency>',
  'filename_example': 'tasmax_NAM_UCSD-SIO_LOCA2-1_CMIP6_historical_ACCESS-CM2_r1i1p1f1_day_19500102-19500131.nc'}}

### process frequency

In [4]:
frequency.pop("version_metadata")
keyList = list(frequency["frequency"].keys())
keepKey = ["day"]
for count, key in enumerate(keyList):
    if key not in keepKey:
        frequency["frequency"].pop(key)
frequency

{'frequency': {'day': 'daily mean samples'}}

### process grid_label

In [5]:
grid_label.pop("version_metadata")
keyList = list(grid_label["grid_label"].keys())
keepKey = ["gn", "gr", "gr1"]
for count, key in enumerate(keyList):
    if key not in keepKey:
        grid_label["grid_label"].pop(key)
grid_label["grid_label"]["gn"] = grid_label["grid_label"]["gn"].replace(
    "a model's ", "the analysis-"
)
grid_label

{'grid_label': {'gn': 'data reported on the analysis-native grid',
  'gr': "regridded data reported on the data provider's preferred target grid",
  'gr1': 'regridded data reported on a grid other than the native grid and other than the preferred target grid'}}

### create institution_id

In [6]:
institution_id = {}
institution_id["institution_id"] = {}
institution_id["institution_id"]["UCM-ACSL"] = {}
institution_id["institution_id"]["UCM-ACSL"][
    "contact"
] = "John T. Abatzoglou; jabatzoglou@ucmerced.edu"
institution_id["institution_id"]["UCM-ACSL"][
    "name"
] = "Applied Climate Science Laboratory, University of California, Merced, 5200 N. Lake Road, Merced, CA 95343, USA"
institution_id["institution_id"]["UCM-ACSL"]["ROR"] = "00d9ah105"
institution_id["institution_id"]["UCM-ACSL"]["URL"] = "https://www.climatologylab.org"
institution_id["institution_id"]["UCSD-SIO"] = {}
institution_id["institution_id"]["UCSD-SIO"][
    "contact"
] = "Daniel Feldman; DRFeldman@lbl.gov"
institution_id["institution_id"]["UCSD-SIO"][
    "name"
] = "Scripps Institution of Oceanography, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA"
institution_id["institution_id"]["UCSD-SIO"]["ROR"] = "04v7hvq31"
institution_id["institution_id"]["UCSD-SIO"]["URL"] = "https://scripps.ucsd.edu"
institution_id["institution_id"]["TTU"] = {}
institution_id["institution_id"]["TTU"][
    "contact"
] = "Katharine Hayhoe; katharine.hayhoe@ttu.edu"
institution_id["institution_id"]["TTU"][
    "name"
] = "Texas Tech University, 2520 Broadway Avenue, Lubbock, TX 79409, USA"
institution_id["institution_id"]["TTU"]["ROR"] = "0405mnx93"
institution_id["institution_id"]["TTU"]["URL"] = "https://www.ttu.edu/"
institution_id

{'institution_id': {'UCM-ACSL': {'contact': 'John T. Abatzoglou; jabatzoglou@ucmerced.edu',
   'name': 'Applied Climate Science Laboratory, University of California, Merced, 5200 N. Lake Road, Merced, CA 95343, USA',
   'ROR': '00d9ah105',
   'URL': 'https://www.climatologylab.org'},
  'UCSD-SIO': {'contact': 'Daniel Feldman; DRFeldman@lbl.gov',
   'name': 'Scripps Institution of Oceanography, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA',
   'ROR': '04v7hvq31',
   'URL': 'https://scripps.ucsd.edu'},
  'TTU': {'contact': 'Katharine Hayhoe; katharine.hayhoe@ttu.edu',
   'name': 'Texas Tech University, 2520 Broadway Avenue, Lubbock, TX 79409, USA',
   'ROR': '0405mnx93',
   'URL': 'https://www.ttu.edu/'}}}

### process license

In [ ]:
# cleanup upstream metadata
license.pop("version_metadata")
license["license"].pop("license")
# remap license_template
license["license"]["license_template"] = " ".join(
    [
        "<license_id>; DRDCP data produced by <institution_id>",
        "is licensed under a <license_type> License (<license_url>).",
        "Consult https://pcmdi.llnl.gov/CMIP6/TermsOfUse",
        "for terms of use governing DRDCP output, including",
        "citation requirements and proper acknowledgment.",
        "The data producers and data providers make no",
        "warranty, either express or implied, including,",
        "but not limited to, warranties of merchantability",
        "and fitness for a particular purpose. All",
        "liabilities arising from the supply of the",
        "information (including any liability arising",
        "in negligence) are excluded to the fullest",
        "extent permitted by law.",
    ]
)

# remap license_options to license_id
license["license"]["license_id"] = {}
# CC BY 4.0
license["license"]["license_id"]["CC BY 4.0"] = {}
license["license"]["license_id"]["CC BY 4.0"]["license_type"] = license["license"][
    "license_options"
]["CC BY 4.0"]["license_id"]
license["license"]["license_id"]["CC BY 4.0"]["license_url"] = license["license"][
    "license_options"
]["CC BY 4.0"]["license_url"]
# CC0 1.0
license["license"]["license_id"]["CC0 1.0"] = {}
license["license"]["license_id"]["CC0 1.0"]["license_type"] = license["license"][
    "license_options"
]["CC0 1.0"]["license_id"]
license["license"]["license_id"]["CC0 1.0"]["license_url"] = license["license"][
    "license_options"
]["CC0 1.0"]["license_url"]
# cleanup
license["license"].pop("license_options")
# print license
license

{'license': {'license_template': '<license_id>; DRDCP data produced by <Your Institution; see DRDCP_institution_id.json> is licensed under a <Creative Commons; select and insert a license_id; see below> License (<insert the matching license_url; see below>). Consult https://pcmdi.llnl.gov/CMIP6/TermsOfUse for terms of use governing DRDCP output, including citation requirements and proper acknowledgment. The data producers and data providers make no warranty, either express or implied, including, but not limited to, warranties of merchantability and fitness for a particular purpose. All liabilities arising from the supply of the information (including any liability arising in negligence) are excluded to the fullest extent permitted by law.',
  'license_id': {'CC BY 4.0': {'license_type': 'Creative Commons Attribution 4.0 International',
    'license_url': 'https://creativecommons.org/licenses/by/4.0/'},
   'CC0 1.0': {'license_type': 'Creative Commons CC0 1.0 Universal Public Domain Ded

In [8]:
### process nominal_resolution

In [9]:
nominal_resolution.pop("version_metadata")
nominal_resolution

{'nominal_resolution': {'0.5 km': 'Resolution of 0.5 km',
  '10000 km': 'Resolution of 10000 km',
  '1000 km': 'Resolution of 1000 km',
  '100 km': 'Resolution of 100 km',
  '10 km': 'Resolution of 10 km',
  '1 km': 'Resolution of 1 km',
  '1x1 degree': 'Resolution of 1x1 degree',
  '2.5 km': 'Resolution of 2.5 km',
  '2500 km': 'Resolution of 2500 km',
  '250 km': 'Resolution of 250 km',
  '25 km': 'Resolution of 25 km',
  '5000 km': 'Resolution of 5000 km',
  '500 km': 'Resolution of 500 km',
  '50 km': 'Resolution of 50 km',
  '5 km': 'Resolution of 5 km'}}

### create product

In [10]:
product = {}
product["product"] = {}
product["product"] = ["downscaled-dynamical", "downscaled-statistical"]
product

{'product': ['downscaled-dynamical', 'downscaled-statistical']}

### process realm

In [11]:
realm.pop("version_metadata")
keyList = list(realm["realm"].keys())
keepKeys = ["atmos", "land"]
for count, key in enumerate(keyList):
    if key not in keepKeys:
        realm["realm"].pop(key)
realm

{'realm': {'atmos': 'Atmosphere', 'land': 'Land Surface and Subsurface'}}

### create region

In [ ]:
region = {}
region["region"] = [
    "north_america",
]
region

{'region': ['north_america']}

### create required_global_attributes

In [13]:
required_global_attributes = {}
required_global_attributes["required_global_attributes"] = [
    "Conventions",
    "activity_id",
    "calendar",
    "contact",
    "creation_date",
    "data_specs_version",
    "driving_activity_id",
    "driving_experiment_id",
    "driving_mip_era",
    "driving_source_id",
    "driving_variant_label",
    "frequency",
    "grid",
    "grid_label",
    "license",
    "license_id",
    "license_url",
    "nominal_resolution",
    "product",
    "realm",
    "region",
    "region_id",
    "table_id",
    "tracking_id",
    "variable_id",
]
required_global_attributes

{'required_global_attributes': ['Conventions',
  'activity_id',
  'calendar',
  'contact',
  'creation_date',
  'data_specs_version',
  'driving_activity_id',
  'driving_experiment_id',
  'driving_mip_era',
  'driving_source_id',
  'driving_variant_label',
  'frequency',
  'grid',
  'grid_label',
  'license',
  'license_id',
  'license_url',
  'nominal_resolution',
  'product',
  'realm',
  'region',
  'region_id',
  'table_id',
  'tracking_id',
  'variable_id']}

### create source_id

In [14]:
source_id = {}
source_id["source_id"] = {}
# LOCA2-0
source_id["source_id"]["LOCA2-0"] = {}
source_id["source_id"]["LOCA2-0"]["calendar"] = "gregorian"
source_id["source_id"]["LOCA2-0"]["contact"] = "Daniel Feldman; DRFeldman@lbl.gov"
source_id["source_id"]["LOCA2-0"]["further_info_url"] = "https://loca.ucsd.edu/"
source_id["source_id"]["LOCA2-0"]["grid"] = "5 x 5 km latitude x longitude"
source_id["source_id"]["LOCA2-0"]["grid_label"] = "gn"
source_id["source_id"]["LOCA2-0"]["institution_id"] = "UCSD-SIO"
source_id["source_id"]["LOCA2-0"][
    "license"
] = "Creative Commons Attribution 4.0 International"
source_id["source_id"]["LOCA2-0"]["license_id"] = "CC BY 4.0"
source_id["source_id"]["LOCA2-0"][
    "license_url"
] = "https://creativecommons.org/licenses/by/4.0/"
source_id["source_id"]["LOCA2-0"]["nominal_resolution"] = "5 km"
source_id["source_id"]["LOCA2-0"]["product"] = "downscaled-statistical"
source_id["source_id"]["LOCA2-0"]["reference"] = " ".join(
    [
        "Pierce, David W., Daniel R. Cayan, and Bridget L. Thrasher",
        "(2014) Statistical downscaling using Localized Constructed",
        "Analogs (LOCA). Journal of Hydrometeorology, 15 (6), pp",
        "2558-2585. https://doi.org/10.1175/JHM-D-14-0082.1",
    ]
)
source_id["source_id"]["LOCA2-0"]["region"] = "north_america"
source_id["source_id"]["LOCA2-0"]["region_id"] = "NAM"
source_id["source_id"]["LOCA2-0"][
    "source"
] = "LOCA 2.0: Statistically-downscaled climate model projections based on CMIP6"
source_id["source_id"]["LOCA2-0"]["source_name"] = "LOCA"
source_id["source_id"]["LOCA2-0"]["source_version"] = "2.0"
source_id["source_id"]["LOCA2-0"]["title"] = "LOCA 2.0 dataset prepared for DRCDP"
# LOCA2-1
source_id["source_id"]["LOCA2-1"] = {}
source_id["source_id"]["LOCA2-1"]["calendar"] = "gregorian"
source_id["source_id"]["LOCA2-1"]["contact"] = "Daniel Feldman; DRFeldman@lbl.gov"
source_id["source_id"]["LOCA2-1"]["further_info_url"] = "https://loca.ucsd.edu/"
source_id["source_id"]["LOCA2-1"]["grid"] = "5 x 5 km latitude x longitude"
source_id["source_id"]["LOCA2-1"]["grid_label"] = "gn"
source_id["source_id"]["LOCA2-1"]["institution_id"] = "UCSD-SIO"
source_id["source_id"]["LOCA2-1"][
    "license"
] = "Creative Commons Attribution 4.0 International"
source_id["source_id"]["LOCA2-1"]["license_id"] = "CC BY 4.0"
source_id["source_id"]["LOCA2-1"][
    "license_url"
] = "https://creativecommons.org/licenses/by/4.0/"
source_id["source_id"]["LOCA2-1"]["nominal_resolution"] = "5 km"
source_id["source_id"]["LOCA2-1"]["product"] = "downscaled-statistical"
source_id["source_id"]["LOCA2-1"]["reference"] = " ".join(
    [
        "Pierce, David W., Daniel R. Cayan, and Bridget L. Thrasher",
        "(2014) Statistical downscaling using Localized Constructed",
        "Analogs (LOCA). Journal of Hydrometeorology, 15 (6), pp",
        "2558-2585. https://doi.org/10.1175/JHM-D-14-0082.1",
    ]
)
source_id["source_id"]["LOCA2-1"]["region"] = "north_america"
source_id["source_id"]["LOCA2-1"]["region_id"] = "NAM"
source_id["source_id"]["LOCA2-1"][
    "source"
] = "LOCA 2.1: Statistically-downscaled climate model projections based on CMIP6"
source_id["source_id"]["LOCA2-1"]["source_name"] = "LOCA"
source_id["source_id"]["LOCA2-1"]["source_version"] = "2.1"
source_id["source_id"]["LOCA2-1"]["title"] = "LOCA 2.1 dataset prepared for DRCDP"
# MACA3-0
source_id["source_id"]["MACA3-0"] = {}
source_id["source_id"]["MACA3-0"]["calendar"] = "gregorian"
source_id["source_id"]["MACA3-0"][
    "contact"
] = "John T. Abatzoglou; jabatzoglou@ucmerced.edu"
source_id["source_id"]["MACA3-0"][
    "further_info_url"
] = "https://www.climatologylab.org/maca.html"
source_id["source_id"]["MACA3-0"]["grid"] = "10 x 10 km latitude x longitude"
source_id["source_id"]["MACA3-0"]["grid_label"] = "gn"
source_id["source_id"]["MACA3-0"]["institution_id"] = "UCM-ACSL"
source_id["source_id"]["MACA3-0"][
    "license"
] = "Creative Commons CC0 1.0 Universal Public Domain Dedication"
source_id["source_id"]["MACA3-0"][
    "license_id"
] = "CC0 1.0"  # https://www.climatologylab.org/maca.html#References
source_id["source_id"]["MACA3-0"][
    "license_url"
] = "https://creativecommons.org/publicdomain/zero/1.0/"
source_id["source_id"]["MACA3-0"]["nominal_resolution"] = "10 km"
source_id["source_id"]["MACA3-0"]["product"] = "downscaled-statistical"
source_id["source_id"]["MACA3-0"]["reference"] = " ".join(
    [
        "Abatzoglou, John T., and Timothy J. Brown",
        "(2012) A comparison of statistical downscaling",
        "methods suited for wildfire applications.",
        "International Journal of Climatology, 32 (5),",
        "pp 772-780. https://doi.org/10.1002/joc.2312",
    ]
)
source_id["source_id"]["MACA3-0"]["region"] = "north_america"
source_id["source_id"]["MACA3-0"]["region_id"] = "NAM"
source_id["source_id"]["MACA3-0"][
    "source"
] = "MACA 3.0: Statistically-downscaled climate model projections based on CMIP6"
source_id["source_id"]["MACA3-0"]["source_name"] = "MACA"
source_id["source_id"]["MACA3-0"]["source_version"] = "3.0"
source_id["source_id"]["MACA3-0"]["title"] = "MACA 3.0 dataset prepared for DRCDP"
# STAR-ESDM1-0
source_id["source_id"]["STAR-ESDM1-0"] = {}

source_id["source_id"]["STAR-ESDM1-0"]["calendar"] = "365_day"
source_id["source_id"]["STAR-ESDM1-0"][
    "contact"
] = "Katharine Hayhoe; katharine.hayhoe@ttu.edu"
source_id["source_id"]["STAR-ESDM1-0"][
    "further_info_url"
] = "https://www.depts.ttu.edu/csc/data/"
source_id["source_id"]["STAR-ESDM1-0"]["grid"] = "5 x 5 km latitude x longitude"
source_id["source_id"]["STAR-ESDM1-0"]["grid_label"] = "gn"
source_id["source_id"]["STAR-ESDM1-0"]["institution_id"] = "TTU"
source_id["source_id"]["STAR-ESDM1-0"][
    "license"
] = "Creative Commons Attribution 4.0 International"
source_id["source_id"]["STAR-ESDM1-0"]["license_id"] = "CC BY 4.0"
source_id["source_id"]["STAR-ESDM1-0"][
    "license_url"
] = "https://creativecommons.org/licenses/by/4.0/"
source_id["source_id"]["STAR-ESDM1-0"]["nominal_resolution"] = "5 km"
source_id["source_id"]["STAR-ESDM1-0"]["product"] = "downscaled-statistical"
source_id["source_id"]["STAR-ESDM1-0"]["reference"] = " ".join(
    [
        "Hayhoe, K., Scott-Fleming, I., Stoner, A., and",
        "Wuebbles, D. J. (2024) STAR-ESDM: A generalizable",
        "approach to generating high-resolution climate",
        "projections through signal decomposition.",
        "Earth's Future, 12, e2023EF004107.",
        "https://doi.org/10.1029/2023EF004107",
    ]
)
source_id["source_id"]["STAR-ESDM1-0"]["region"] = "north_america"
source_id["source_id"]["STAR-ESDM1-0"]["region_id"] = "NAM"
source_id["source_id"]["STAR-ESDM1-0"][
    "source"
] = "STAR-ESDM 1.0: Statistically-downscaled climate model projections based on CMIP6"
source_id["source_id"]["STAR-ESDM1-0"]["source_name"] = "STAR-ESDM"
source_id["source_id"]["STAR-ESDM1-0"]["source_version"] = "1.0"
source_id["source_id"]["STAR-ESDM1-0"][
    "title"
] = "STAR-ESDM 1.0 dataset prepared for DRCDP"
source_id

{'source_id': {'LOCA2-0': {'calendar': 'gregorian',
   'contact': 'Daniel Feldman; DRFeldman@lbl.gov',
   'further_info_url': 'https://loca.ucsd.edu/',
   'grid': '5 x 5 km latitude x longitude',
   'grid_label': 'gn',
   'institution_id': 'UCSD-SIO',
   'license': 'Creative Commons Attribution 4.0 International',
   'license_id': 'CC BY 4.0',
   'license_url': 'https://creativecommons.org/licenses/by/4.0/',
   'nominal_resolution': '5 km',
   'product': 'downscaled-statistical',
   'reference': 'Pierce, David W., Daniel R. Cayan, and Bridget L. Thrasher (2014) Statistical downscaling using Localized Constructed Analogs (LOCA). Journal of Hydrometeorology, 15 (6), pp 2558-2585. https://doi.org/10.1175/JHM-D-14-0082.1',
   'region': 'north_america',
   'region_id': 'NAM',
   'source': 'LOCA 2.0: Statistically-downscaled climate model projections based on CMIP6',
   'source_name': 'LOCA',
   'source_version': '2.0',
   'title': 'LOCA 2.0 dataset prepared for DRCDP'},
  'LOCA2-1': {'calen

### create table_id

In [15]:
table_id = {}
table_id["table_id"] = ["APday"]
table_id

{'table_id': ['APday']}

### create tracking_id

In [16]:
tracking_id = {}
tracking_id["tracking_id_prefix"] = ["hdl:21.14100"]
tracking_id

{'tracking_id_prefix': ['hdl:21.14100']}

### write all files out to repo root dir

In [17]:
files = [
    "DRS",
    "frequency",
    "grid_label",
    "institution_id",
    "license",
    "nominal_resolution",
    "product",
    "realm",
    "region",
    "required_global_attributes",
    "source_id",
    "table_id",
    "tracking_id",
]
for count, name in enumerate(files):
    print(count, name)
    dic = eval(name)
    # write file
    outFile = "".join(["../DRCDP_", name, ".json"])
    with open(outFile, "w") as f:
        json.dump(
            dic, f, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":")
        )

0 DRS
1 frequency
2 grid_label
3 institution_id
4 license
5 nominal_resolution
6 product
7 realm
8 region
9 required_global_attributes
10 source_id
11 table_id
12 tracking_id


### create composite DRCDP_CV.json for CMOR

In [25]:
%%time
CV = {}
CV["CV"] = {}
CV["CV"]["DRS"] = DRS["DRS"]
CV["CV"]["activity_id"] = "DRCDP"
CV["CV"]["frequency"] = frequency["frequency"]
CV["CV"]["grid_label"] = grid_label["grid_label"]
CV["CV"]["institution_id"] = institution_id["institution_id"]
CV["CV"]["license"] = license["license"]
CV["CV"]["nominal_resolution"] = nominal_resolution["nominal_resolution"]
CV["CV"]["product"] = product["product"]
CV["CV"]["realm"] = realm["realm"]
CV["CV"]["region"] = region["region"]
CV["CV"]["required_global_attributes"] = required_global_attributes[
    "required_global_attributes"
]
CV["CV"]["source_id"] = source_id["source_id"]
CV["CV"]["table_id"] = table_id["table_id"]
CV["CV"]["tracking_id_prefix"] = tracking_id["tracking_id_prefix"]
# CV
# write file
outFile = "".join(["../Tables/DRCDP_CV.json"])
with open(outFile, "w") as f:
    json.dump(CV, f, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"))

CPU times: user 26 μs, sys: 2 μs, total: 28 μs
Wall time: 34.1 μs
